### `Parte_1: Limpieza de datos de Alumnos Prioritarios, Preferentes y/o Beneficiarios`

En esta sección, nos enfocaremos en la limpieza de datos específicos relacionados con el registro de Alumnos Prioritarios, Preferentes y/o Beneficiarios de la Subvención. Este proceso abarcará:

#### Bases de datos involucradas:

1. **Esquema de registro de Alumnos Prioritarios, Preferentes y/o Beneficiarios:**

#### Objetivos del Notebook:

En este notebook, llevaremos a cabo las siguientes tareas:

- **Limpieza de registro de Alumnos Prioritarios, Preferentes y/o Beneficiarios:**
  Se verificara verifica que los valores de este base de datos seas razonables y posteriormente se realiza el merge con los dataframes de  rendimiento y asistencia 

- **Generación de un archivo combinado con datos limpios:**
  Adicionalmente, se creará un archivo nuevo que contendrá los datos limpios provenientes de las bases de datos previamente tratadas.

In [1]:
# importacion de librerias 
import os 
import pandas as pd 

In [2]:
# Definir la ruta del archivo ' abrevie Preferentes_Prioritarios_y_Beneficiarios como PPB y Resumen_Rendimiento como RR 
ruta_PPB = os.path.join('data', 'Preferentes_Prioritarios_y_Beneficiarios_2022.csv')
ruta_Asistencia= os.path.join('output_data', 'asistencia.csv')
ruta_rendimiento= os.path.join('output_data', 'rendimiento_estudiantes.csv')


### `Verificar valores atípicos y Renombrar datos de Preferentes_Prioritarios_y_Beneficiarios.`

* las columnas a tomar en cuenta se describen a continuación

| Columna          | Descripción                                             |
|------------------|---------------------------------------------------------|
| MRUN             | Identificador único del estudiante (máscara del RUN)     |
| PRIORITARIO_ALU  | Indicador de alumno prioritario (0: No prioritario, 1: prioritario)           |
| PREFERENTE_ALU   | Indicador de alumno preferente (0: No preferente, 1: Preferente) |
| BEN_SEP          | Indicador de alumno beneficiario de la SEP (0: No beneficiario SEP, 1: Beneficiario SEP ) |
| CRITERIO_SEP     | Criterio por el cual se considera prioritario: <br>0: No prioritario (No cumple requisito o Puntaje Alto)<br>1: Pertenece a Chile Solidario<br>2: Puntaje Ficha de Protección Social menor o igual al punto de corte<br>3: Otro criterio |



In [3]:
# dataframe alumnos Preferentes_Prioritarios_y_Beneficiarios
df_PPB = pd.read_csv(ruta_PPB, sep=';')

df_PPB = df_PPB[['MRUN', 'CRITERIO_SEP', 'BEN_SEP', 'PREFERENTE_ALU', 'PRIORITARIO_ALU']]

/var/folders/rn/8kg7t05x7l3fy1mwbgcdkw240000gp/T/ipykernel_25639/728172888.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_PPB = pd.read_csv(ruta_PPB, sep=';')


In [4]:
# aplicamos una funcion que verifica que los datos de las columna seleccionada no difieran de 1 o 0

def verificar_valores_atipicos(df, column):
    '''
    Retorna True si los valores difieren de 0 o 1 y False en caso contrario 
    
    '''
    hay_valor_distinto = (df[column] != 0) & (df[column] != 1)
    
    resultado = hay_valor_distinto.any()
    
    return resultado

print(verificar_valores_atipicos(df_PPB,'BEN_SEP'))

print(verificar_valores_atipicos(df_PPB, 'PREFERENTE_ALU' ))

print(verificar_valores_atipicos(df_PPB,'PRIORITARIO_ALU'))



False
False
False


In [5]:
#renombramos la columna MRUN para mayor legebilidad 
df_PPB = df_PPB.rename(columns={'MRUN': 'RUN_ALUMNO'})

# Reemplazar los valores en la columna 'PRIORITARIO_ALU'
df_PPB['PRIORITARIO_ALU'] = df_PPB['PRIORITARIO_ALU'].replace({0: 'No prioritario', 1: 'prioritario'})
# Reemplazar los valores en la columna 'PREFERENTE_ALU'
df_PPB['PREFERENTE_ALU'] = df_PPB['PREFERENTE_ALU'].replace({0: 'No preferente', 1: 'Preferente'})
# Reemplazar los valores en la columna 'BEN_SEP'
df_PPB['BEN_SEP'] = df_PPB['BEN_SEP'].replace({0: 'No beneficiario SEP', 1: 'Beneficiario SEP'})

In [6]:
df_PPB.head()

,RUN_ALUMNO,CRITERIO_SEP,BEN_SEP,PREFERENTE_ALU,PRIORITARIO_ALU
0,25,0,No beneficiario SEP,Preferente,No prioritario
1,31,1,Beneficiario SEP,No preferente,prioritario
2,47,2,No beneficiario SEP,No preferente,prioritario
3,48,2,Beneficiario SEP,No preferente,prioritario
4,86,1,Beneficiario SEP,No preferente,prioritario


### `Realizamos el merge con Asistencia.csv y Rendimiento.csv ( datos limpios )`

* realizar el merge por la columna 'RUN_ALUMNO'
* depositar el dataframe resultante e un archivo.csv

In [7]:
df_asistencia = pd.read_csv(ruta_Asistencia, sep=',')

df_rendimiento =  pd.read_csv(ruta_rendimiento, sep=',')

In [8]:
df_asistencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696513 entries, 0 to 696512
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   MES                      696513 non-null  int64  
 1   COD_PRO_RBD              696513 non-null  int64  
 2   COD_COM_RBD              696513 non-null  int64  
 3   RUN_ALUMNO               696513 non-null  int64  
 4   GENERO_ALUMNO            696513 non-null  int64  
 5   FECHA_NACIMIENTO_ALUMNO  696513 non-null  int64  
 6   NOMBRE_COMUNA_ALUMNO     696513 non-null  object 
 7   COD_ENSE2                696513 non-null  int64  
 8   DIAS_ASISTIDOS           696513 non-null  int64  
 9   ASISTENCIA_PROMEDIO      696513 non-null  float64
dtypes: float64(1), int64(8), object(1)
memory usage: 53.1+ MB


In [9]:
df_rendimiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3092848 entries, 0 to 3092847
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   RUN_ALUMNO  int64  
 1   RBD         int64  
 2   COD_JOR     int64  
 3   PROM_GRAL   float64
 4   SIT_FIN_R   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 118.0+ MB


In [10]:
df_PPB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999491 entries, 0 to 2999490
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   RUN_ALUMNO       int64 
 1   CRITERIO_SEP     int64 
 2   BEN_SEP          object
 3   PREFERENTE_ALU   object
 4   PRIORITARIO_ALU  object
dtypes: int64(2), object(3)
memory usage: 114.4+ MB


In [12]:
merged_df = pd.merge(df_asistencia, df_PPB, on='RUN_ALUMNO')

merged_df_estudiantes =  pd.merge(merged_df, df_rendimiento, on='RUN_ALUMNO')

merged_df_estudiantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480128 entries, 0 to 480127
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   MES                      480128 non-null  int64  
 1   COD_PRO_RBD              480128 non-null  int64  
 2   COD_COM_RBD              480128 non-null  int64  
 3   RUN_ALUMNO               480128 non-null  int64  
 4   GENERO_ALUMNO            480128 non-null  int64  
 5   FECHA_NACIMIENTO_ALUMNO  480128 non-null  int64  
 6   NOMBRE_COMUNA_ALUMNO     480128 non-null  object 
 7   COD_ENSE2                480128 non-null  int64  
 8   DIAS_ASISTIDOS           480128 non-null  int64  
 9   ASISTENCIA_PROMEDIO      480128 non-null  float64
 10  CRITERIO_SEP             480128 non-null  int64  
 11  BEN_SEP                  480128 non-null  object 
 12  PREFERENTE_ALU           480128 non-null  object 
 13  PRIORITARIO_ALU          480128 non-null  object 
 14  RBD 

In [13]:
merged_df_estudiantes

,MES,COD_PRO_RBD,COD_COM_RBD,RUN_ALUMNO,GENERO_ALUMNO,FECHA_NACIMIENTO_ALUMNO,NOMBRE_COMUNA_ALUMNO,COD_ENSE2,DIAS_ASISTIDOS,ASISTENCIA_PROMEDIO,CRITERIO_SEP,BEN_SEP,PREFERENTE_ALU,PRIORITARIO_ALU,RBD,COD_JOR,PROM_GRAL,SIT_FIN_R
0,8,31,3101,2112520,2,200901,COPIAPÓ,7,14,0.700000,2,Beneficiario SEP,No preferente,prioritario,423,1,6.2,P
1,8,31,3101,3962434,1,200901,COPIAPÓ,7,12,0.600000,0,Beneficiario SEP,Preferente,No prioritario,420,3,4.9,P
2,8,31,3101,4199164,1,200707,COPIAPÓ,7,10,0.500000,2,Beneficiario SEP,No preferente,prioritario,403,1,5.2,P
3,8,31,3101,4218504,2,200812,COPIAPÓ,7,15,0.750000,2,Beneficiario SEP,No preferente,prioritario,430,3,6.6,P
4,8,31,3101,6990009,2,200903,COPIAPÓ,7,15,0.750000,2,Beneficiario SEP,No preferente,prioritario,430,3,6.7,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480123,8,131,13130,24338899,1,201101,SAN MIGUEL,2,21,1.000000,2,Beneficiario SEP,No preferente,prioritario,41794,1,5.3,P
480124,8,131,13130,25757929,2,201110,SAN MIGUEL,2,20,0.952381,0,Beneficiario SEP,Preferente,No prioritario,41794,1,6.3,P
480125,8,131,13130,26254392,1,201106,SAN MIGUEL,2,19,0.904762,0,Beneficiario SEP,Preferente,No prioritario,11906,2,6.3,P
480126,8,131,13130,26305268,2,201112,SAN MIGUEL,2,21,1.000000,0,Beneficiario SEP,Preferente,No prioritario,41794,1,6.6,P


In [14]:
merged_df_estudiantes.to_csv('output_data/info_por_estudiante.csv', index=False)